In [1]:
from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to your setup
NEO4J_URI = "bolt://neo4j:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "Bienvenue2024!"
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())

2.8.0


In [14]:
# Create a minimal example graph
gds.run_cypher(
  """
  CREATE
  (m: City {name: "Malmö"}),
  (l: City {name: "London"}),
  (s: City {name: "San Mateo"}),
  (m)-[:FLY_TO]->(l),
  (l)-[:FLY_TO]->(m),
  (l)-[:FLY_TO]->(s),
  (s)-[:FLY_TO]->(l)
  """
)

# Project the graph into the GDS Graph Catalog
# We call the object representing the projected graph `G_office`
G_office, project_result = gds.graph.project("neo4j-offices1", "City", "FLY_TO")

# Run the mutate mode of the PageRank algorithm
mutate_result = gds.pageRank.mutate(G_office, tolerance=0.5, mutateProperty="rank")


In [17]:
# We can inspect the node properties of our projected graph directly
# via the graph object and see that indeed the new property exists
#assert G_office.node_properties("City") == ["rank"]
def check_node_properties():
    try:
        if G_office.node_properties("City") == ["rank"]:
            print("True")
    except AssertionError :
        print("Error")
        
check_node_properties()

True


In [18]:
## Projecting a graph object
# We put this simple graph in our database
gds.run_cypher(
  """
  CREATE
    (m: City {name: "Malmö"}),
    (l: City {name: "London"}),
    (s: City {name: "San Mateo"}),

    (m)-[:FLY_TO]->(l),
    (l)-[:FLY_TO]->(m),
    (l)-[:FLY_TO]->(s),
    (s)-[:FLY_TO]->(l)
  """
)

# We estimate required memory of the operation
res = gds.graph.project.estimate(
    ["City"],                   #  Node projection
    "FLY_TO",                   #  Relationship projection
    readConcurrency=4           #  Configuration parameters
)

In [19]:
# assert res["bytesMax"] < 1e12
def check_res():
    try:
        if res["bytesMax"] < 1e12:
            print("True")
    except AssertionError :
        print("Error")
        
check_res()

True


In [21]:
G, result = gds.graph.project(
    "offices1",                  #  Graph name
    ["City"],                   #  Node projection
    "FLY_TO",                   #  Relationship projection
    readConcurrency=4           #  Configuration parameters
)


In [22]:
# assert G.node_count() == result["nodeCount"]
def check_result():
    try:
        if G.node_count() == result["nodeCount"]:
            print("True")
    except AssertionError :
        print("Error")
        
check_result()

True


In [9]:
G = gds.graph.get("offices")
print(G)

Graph(name=offices, node_count=6, relationship_count=8)


In [24]:
## Constructing a graph from DataFrames
import pandas as pd
nodes = pd.DataFrame(
    {
        "nodeId": [0, 1, 2, 3],
        "labels":  ["A", "B", "C", "A"],
        "prop1": [42, 1337, 8, 0],
        "otherProperty": [0.1, 0.2, 0.3, 0.4]
    }
)

relationships = pd.DataFrame(
    {
        "sourceNodeId": [0, 1, 2, 3],
        "targetNodeId": [1, 2, 3, 0],
        "relationshipType": ["REL", "REL", "REL", "REL"],
        "weight": [0.0, 0.0, 0.1, 42.0]
    }
)

G = gds.graph.construct(
    "my-graph3",      # Graph name
    nodes,           # One or more dataframes containing node data
    relationships    # One or more dataframes containing relationship data
)

assert "REL" in G.relationship_types()

In [25]:
# Function to check and print relationship type
def check_relationship_type():
    try:
        if "REL" in G.relationship_types():
            print("Relationship type 'REL' exists.")
        else:
            print("Relationship type 'REL' does not exist.")
    except AssertionError:
        print("Assertion failed: 'REL' is not in the list of relationship types.")
# Check and print the relationship type
check_relationship_type()

Relationship type 'REL' exists.


In [28]:
gds.run_cypher(
    """
    CREATE
    (a:A {prop1: 42,    otherProperty: 0.1}),
    (b:B {prop1: 1337,  otherProperty: 0.2}),
    (c:C {prop1: 8,     otherProperty: 0.3}),
    (d:A {prop1: 0,     otherProperty: 0.4}),
    (a)-[:REL {weight: 0.0}]->(b),
    (b)-[:REL {weight: 0.0}]->(c),
    (c)-[:REL {weight: 0.1}]->(d),
    (d)-[:REL {weight: 42.0}]->(a)
    """
)

""


In [3]:
pip install boto3

  Using cached boto3-1.34.153-py3-none-any.whl (139 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached botocore-1.34.153-py3-none-any.whl (12.4 MB)
  Using cached s3transfer-0.10.2-py3-none-any.whl (82 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import boto3

# boto3 is the AWS SDK library for Python.
# We can use the low-level client to make API calls to DynamoDB.
client = boto3.client(
    'dynamodb',
    region_name='eu-west-3',
    #aws_access_key_id='AKIA4MTWHWW4ZVPEBXYPGMFH',
    #aws_secret_access_key='bOrcX8ltPNaOHFnlj60Xa8ouge6GutPAVvGiZK7fovZC'
    #aws_session_token='your_session_token'  # Only if using temporary session tokens

)

In [5]:
try:
    resp = client.create_table(
        TableName="Books1",
        # Declare your Primary Key in the KeySchema argument
        KeySchema=[
            {
                "AttributeName": "Author",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "Title",
                "KeyType": "RANGE"
            }
        ],
        # Any attributes used in KeySchema or Indexes must be declared in AttributeDefinitions
        AttributeDefinitions=[
            {
                "AttributeName": "Author",
                "AttributeType": "S"
            },
            {
                "AttributeName": "Title",
                "AttributeType": "S"
            }
        ],
        # ProvisionedThroughput controls the amount of data you can read or write to DynamoDB per second.
        # You can control read and write capacity independently.
        ProvisionedThroughput={
            "ReadCapacityUnits": 1,
            "WriteCapacityUnits": 1
        }
    )
    print("Table created successfully!")
except Exception as e:
    print("Error creating table:")
    print(e)

Table created successfully!


In [6]:
dynamodb = boto3.resource(
    'dynamodb',
    region_name='eu-west-3',
    #aws_access_key_id='AKIA4MTWHWW4ZVPEBXYPGMFH',
    #aws_secret_access_key='bOrcX8ltPNaOHFnlj60Xa8ouge6GutPAVvGiZK7fovZC'
    #aws_session_token='your_session_token'  # Only if using temporary session tokens
    
)

In [7]:
table = dynamodb.Table('Books1')

with table.batch_writer() as batch:
    batch.put_item(Item={"Author": "John Grisham", "Title": "The Rainmaker",
        "Category": "Suspense", "Formats": { "Hardcover": "J4SUKVGU", "Paperback": "D7YF4FCX" } })
    batch.put_item(Item={"Author": "John Grisham", "Title": "The Firm",
        "Category": "Suspense", "Formats": { "Hardcover": "Q7QWE3U2",
        "Paperback": "ZVZAYY4F", "Audiobook": "DJ9KS9NM" } })
    batch.put_item(Item={"Author": "James Patterson", "Title": "Along Came a Spider",
        "Category": "Suspense", "Formats": { "Hardcover": "C9NR6RJ7",
        "Paperback": "37JVGDZG", "Audiobook": "6348WX3U" } })
    batch.put_item(Item={"Author": "Dr. Seuss", "Title": "Green Eggs and Ham",
        "Category": "Children", "Formats": { "Hardcover": "GVJZQ7JK",
        "Paperback": "A4TFUR98", "Audiobook": "XWMGHW96" } })
    batch.put_item(Item={"Author": "William Shakespeare", "Title": "Hamlet",
        "Category": "Drama", "Formats": { "Hardcover": "GVJZQ7JK",
        "Paperback": "A4TFUR98", "Audiobook": "XWMGHW96" } })

In [13]:
gds.run_cypher(
    """
    CREATE
    (a:A1 {Author: "John Grisham", Title: "The Rainmaker",Category: "Suspense"}),
    (b:B1 {Author: "John Grisham", Title: "The Firm",Category: "Suspense"}),
    (c:C1 {Author: "James Patterson", Title: "Along Came a Spider",Category: "Suspense"}),
    (d:D1 {Author: "Dr. Seuss", Title: "Green Eggs and Ham",Category: "Children"}),
    (a)-[:REL1 {weight: 0.0}]->(b),
    (b)-[:REL1 {weight: 0.0}]->(c),
    (c)-[:REL1 {weight: 0.1}]->(d),
    (d)-[:REL1 {weight: 42.0}]->(a)
    """
)

""


In [14]:
gds.run_cypher(
  """
  CREATE
    (a:City {name: "New York City", settled: 1624}),
    (b:City {name: "Philadelphia", settled: 1682}),
    (c:City:Capital {name: "Washington D.C.", settled: 1790}),
    (d:City {name: "Baltimore"}),
    (e:City {name: "Atlantic City"}),
    (f:City {name: "Boston"}),

    (a)-[:ROAD {cost: 50}]->(b),
    (a)-[:ROAD {cost: 50}]->(c),
    (a)-[:ROAD {cost: 100}]->(d),
    (b)-[:ROAD {cost: 40}]->(d),
    (c)-[:ROAD {cost: 40}]->(d),
    (c)-[:ROAD {cost: 80}]->(e),
    (d)-[:ROAD {cost: 30}]->(e),
    (d)-[:ROAD {cost: 80}]->(f),
    (e)-[:ROAD {cost: 40}]->(f);
  """
)
G, project_result = gds.graph.project("road_graph", "City", {"ROAD": {"properties": ["cost"]}})


In [ ]:
# assert G.relationship_count() == 9
def check_relationship_count():
    try:
        if G.relationship_count() == 9:
            print("True")
    except AssertionError :
        print("Error")
        
check_relationship_count()